In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import seaborn.objects as so

In [136]:
raw_data = pd.read_csv(f'/Users/michaelschaid/GitHub/credit_modeling/data/loans_2007.csv', low_memory=False)
raw_data.dtypes

id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

In [205]:
def clean_loans(df):
    
    def find_cols_with_missing_data(_df, threshold):
        """Returns a list of columns with missing data over specified threshold"""
        return [col for col in _df.columns if _df[col].isnull().sum() > (_df.shape[0]*threshold)]
    
        
    ordinal_grade_type = pd.CategoricalDtype(categories = sorted(df.grade.dropna().unique()), ordered = True)
    
    return(
        df
        .iloc[:-2]
        .drop(columns = find_cols_with_missing_data(df, threshold = 0.5))
        .rename(columns = {'term':'term_mo','int_rate':'int_rate_percent' })
        .assign(term_mo  =  lambda df_: df_.term_mo.str.replace(' months', '').dropna().astype(int),
                int_rate_percent = lambda df_: df_.int_rate_percent.str.replace('%','').astype(float),
                grade = lambda df_: df_.grade.astype(ordinal_grade_type),
                emp_title = lambda df_: df_.emp_title.fillna('not_given').str.lower(),
                emp_length = lambda df_: df_.emp_length.str.replace('\D', '', regex = True),
                not_verified = lambda df_: df_.verification_status.str.contains('[nN]ot'),
                renter = lambda df_: df_.home_ownership.str.contrains('RENT')
                
                )
        .drop(columns = ['verification_status', 'sub_grade', 'id', 'member_id'])
        .dropna()

        )
    
loans = clean_loans(raw_data)

    

AttributeError: 'StringMethods' object has no attribute 'contrains'

In [204]:
loans.grade.max()

'G'

In [201]:
raw_data.grade.dropna()

0        B
1        C
2        C
3        C
4        B
        ..
42531    C
42532    B
42533    B
42534    A
42535    A
Name: grade, Length: 42535, dtype: object